Chapter 10: Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

Group operations follow a split-apply-combine sequence. First data is split into groups based on one or more keys (either split across rows or columns), then a function is applied to each group, then the results are combined into a result.

In [2]:
# Start with a tabular dataset made into a DataFrame
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-2.170739,-1.273762
1,a,2,1.088979,1.639102
2,None,1,0.459670,1.101126
3,b,2,-0.266686,0.831452
4,b,1,-0.088097,-0.912940
5,a,<NA>,-0.234249,-0.454648
6,None,1,0.399779,0.819872


In [3]:
# Sort the df by picking out the data1 column and group by key1
grouped = df["data1"]
grouped

0   -2.170739
1    1.088979
2    0.459670
3   -0.266686
4   -0.088097
5   -0.234249
6    0.399779
Name: data1, dtype: float64

In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [5]:
# The groupby object has all the info needed to apply an operation to each of the groups
grouped.mean()

key1
a   -0.438670
b   -0.177392
Name: data1, dtype: float64

In [6]:
# Passing multiple arrays to "group by" gives different groups and different values
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -2.170739
      2       1.088979
b     1      -0.088097
      2      -0.266686
Name: data1, dtype: float64

In [7]:
# You can group by any arrays of the correct length
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005    0.427365
    2006    0.459670
OH  2005   -1.218713
    2006    0.155841
Name: data1, dtype: float64

In [8]:
# size returns a Series containing group sizes
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [9]:
df.groupby("key1").size()

key1
a    3
b    2
dtype: int64

In [10]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [11]:
# The object returned by groupby supports iteration
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -2.170739 -1.273762
1    a     2  1.088979  1.639102
5    a  <NA> -0.234249 -0.454648
b
  key1  key2     data1     data2
3    b     2 -0.266686  0.831452
4    b     1 -0.088097 -0.912940


In [12]:
# This allows for the computation of a dictionary of the data pieces as a one-liner:
pieces = {name: group for name, group in df.groupby("key1")}
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -2.170739 -1.273762
 1    a     2  1.088979  1.639102
 5    a  <NA> -0.234249 -0.454648,
 'b':   key1  key2     data1     data2
 3    b     2 -0.266686  0.831452
 4    b     1 -0.088097 -0.912940}

In [13]:
# We can group the columns of the dataframe by whether they start with "key" or "data"
grouped = df.groupby({"key1": "key", "key2" : "key", "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -2.170739 -1.273762
1  1.088979  1.639102
2  0.459670  1.101126
3 -0.266686  0.831452
4 -0.088097 -0.912940
5 -0.234249 -0.454648
6  0.399779  0.819872
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [14]:
# Compute the mean only for the data2 column and return the result as a dataframe
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -1.273762
     2     1.639102
b    1    -0.912940
     2     0.831452

In [15]:
# Grouping with Dictionaries and Series
# make new df
people = pd.DataFrame(np.random.standard_normal((5, 5)), columns=["a", "b", "c", "d", "e"], index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,-0.365389,0.699102,-1.941074,-1.049131,0.318239
Steve,1.057435,-0.345494,1.374907,0.073339,0.146323
Wanda,-0.495495,-2.337421,-0.770708,0.318820,1.226583
Jill,-0.754938,-2.662357,1.281277,0.014578,-1.205973
Trey,-2.268706,-1.043787,1.244606,-0.765559,-1.298678


In [16]:
# add NA values
people.iloc[2:4, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.365389,0.699102,-1.941074,-1.049131,0.318239
Steve,1.057435,-0.345494,1.374907,0.073339,0.146323
Wanda,-0.495495,NaN,NaN,0.318820,1.226583
Jill,-0.754938,NaN,NaN,0.014578,-1.205973
Trey,-2.268706,-1.043787,1.244606,-0.765559,-1.298678


In [17]:
# Create a dictionary containing a mapping
mapping = {"a": "red", "b": "red", "c": "blue", "d": "blue", "e": "red", "f": "orange"}
mapping
# The mapping "f" above will get ignored

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [18]:
# Pass the dictionary to "groupby"
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

,blue,red
Joe,-2.990205,0.651952
Steve,1.448247,0.858264
Wanda,0.318820,0.731089
Jill,0.014578,-1.960911
Trey,0.479047,-4.611171


In [19]:
# We can also pass the dictionary to a series
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [20]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,1,2
Trey,2,3


In [21]:
# Grouping with Functions

# Group the previous dataframe by passing the len function to groupby
people.groupby(len).sum()

,a,b,c,d,e
3,-0.365389,0.699102,-1.941074,-1.049131,0.318239
4,-3.023644,-1.043787,1.244606,-0.750982,-2.504651
5,0.561940,-0.345494,1.374907,0.392159,1.372907


In [22]:
# You can also pass a mixed function/array/dict/Series to groupby
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-0.365389,0.699102,-1.941074,-1.049131,0.318239
4,two,-2.268706,-1.043787,1.244606,-0.765559,-1.298678
5,one,-0.495495,-0.345494,1.374907,0.073339,0.146323


In [23]:
# Data Aggregation
# An aggregation is a data transformation which produces a scalar from an array (i.e. mean, count, min, sum)
df

,key1,key2,data1,data2
0,a,1,-2.170739,-1.273762
1,a,2,1.088979,1.639102
2,None,1,0.459670,1.101126
3,b,2,-0.266686,0.831452
4,b,1,-0.088097,-0.912940
5,a,<NA>,-0.234249,-0.454648
6,None,1,0.399779,0.819872


In [24]:
# Group by key1
grouped = df.groupby("key1")
# Find 2 smallest values in the grouped df under "data1" column
grouped["data1"].nsmallest(2)

key1   
a     0   -2.170739
      5   -0.234249
b     3   -0.266686
      4   -0.088097
Name: data1, dtype: float64

In [25]:
# You can define your own functions which act on the whole array and pass them to agg
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,3.259718,2.912864
b,1,0.178589,1.744392


In [26]:
# Return to tipping dataset
tips = pd.read_csv("datasets/tips.csv")
print("Shape of data frame: {}".format(tips.shape))
tips.head()

Shape of data frame: (244, 6)


,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [27]:
# Add a tip percentage column
tips["tip_pct"] = tips["tip"]/tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [28]:
# Group the tips by day and smoking status
grouped = tips.groupby(["day", "smoker"])
# Pull the tip percentage column
grouped_pct = grouped["tip_pct"]
# Pass the name of the function as a string to agg
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [29]:
# Can also pass a list of functions
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [30]:
# Can give the returns from the functions new names
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [31]:
# For data frames, can pass the same function to different columns or different functions to different columns
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [32]:
# To apply different functions to different columns, pass a dictionary to agg
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [33]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"], "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [34]:
# Can change the index from grouped keys to a flattened index
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [35]:
# The apply method splits the object it acts on into pieces, invokes a function on each piece, and then concatenates the pieces together.
# Here we want to select the top five tip percentage values by group

# Define a function
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [36]:
# Now group by "smoker" and call apply with the top function
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [37]:
# Functions with optional inputs can also be passed
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [38]:
# Again, you can flatten the indexes
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [39]:
# pandas makes it easy to do quantile analysis on a dataset
# Make new dataset
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.273874,0.759652
1,0.537203,-2.082356
2,1.998605,0.169845
3,0.298119,-2.035682
4,1.318676,-0.075639


In [40]:
# Now find the quartiles for the "data1" column
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0     (-0.0263, 1.692]
1     (-0.0263, 1.692]
2       (1.692, 3.411]
3     (-0.0263, 1.692]
4     (-0.0263, 1.692]
5    (-1.745, -0.0263]
6    (-1.745, -0.0263]
7     (-0.0263, 1.692]
8    (-1.745, -0.0263]
9    (-1.745, -0.0263]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.47, -1.745] < (-1.745, -0.0263] < (-0.0263, 1.692] < (1.692, 3.411]]

In [41]:
# We can then pass the quartiles object to the groupby function
def get_stats(group):
    return pd.DataFrame({"min": group.min(), "max": group.max(), "count": group.count(), "mean": group.mean()})

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                       
(-3.47, -1.745]   data1 -3.463138 -1.749422     52 -2.153619
                  data2 -3.084330  1.964887     52  0.036087
(-1.745, -0.0263] data1 -1.733663 -0.026780    429 -0.656438
                  data2 -2.582544  3.074502    429 -0.005514
(-0.0263, 1.692]  data1 -0.020453  1.691869    482  0.678132
                  data2 -3.280557  4.737723    482  0.050752
(1.692, 3.411]    data1  1.696393  3.410559     37  2.166715
                  data2 -2.149904  1.953695     37 -0.331740

In [42]:
# But note we can do that more easily
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                        min       max count      mean       min       max   
data1                                                                       
(-3.47, -1.745]   -3.463138 -1.749422    52 -2.153619 -3.084330  1.964887   
(-1.745, -0.0263] -1.733663 -0.026780   429 -0.656438 -2.582544  3.074502   
(-0.0263, 1.692]  -0.020453  1.691869   482  0.678132 -3.280557  4.737723   
(1.692, 3.411]     1.696393  3.410559    37  2.166715 -2.149904  1.953695   

                                   
                  count      mean  
data1                              
(-3.47, -1.745]      52  0.036087  
(-1.745, -0.0263]   429 -0.005514  
(-0.0263, 1.692]    482  0.050752  
(1.692, 3.411]       37 -0.331740

In [43]:
# cut returns equal length buckets, qcut returns equal sized buckets
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    2
1    2
2    3
3    2
4    3
Name: data1, dtype: int64

In [44]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.463138 -0.628067    250 -1.295815
      data2 -3.084330  3.006012    250 -0.022070
1     data1 -0.620858  0.006903    250 -0.278934
      data2 -3.023731  3.074502    250  0.023022
2     data1  0.007176  0.695547    250  0.342884
      data2 -3.280557  3.522730    250  0.043483
3     data1  0.701593  3.410559    250  1.285578
      data2 -3.088480  4.737723    250  0.002360

In [46]:
# Fill NA values in a Series or DataFrame
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1    1.406526
2         NaN
3   -1.834898
4         NaN
5    0.522794
dtype: float64

In [47]:
# Fill NA values with the mean of other values
s.fillna(s.mean())

0    0.031474
1    1.406526
2    0.031474
3   -1.834898
4    0.031474
5    0.522794
dtype: float64

In [49]:
# We can also fillna on specific group elements
states= ["Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East", "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio         -0.842303
New York      0.133229
Vermont       1.408741
Florida       1.674211
Oregon        1.102568
Nevada        0.258670
California   -0.731954
Idaho        -0.264024
dtype: float64

In [51]:
# Insert NAN values
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.842303
New York      0.133229
Vermont            NaN
Florida       1.674211
Oregon        1.102568
Nevada             NaN
California   -0.731954
Idaho              NaN
dtype: float64

In [52]:
# Look at some groupby sizes
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [54]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [55]:
data.groupby(group_key).mean()

East    0.321712
West    0.185307
dtype: float64

In [56]:
# Fill NA values with group means
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

Ohio         -0.842303
New York      0.133229
Vermont       0.321712
Florida       1.674211
Oregon        1.102568
Nevada        0.185307
California   -0.731954
Idaho         0.185307
dtype: float64

In [57]:
# We can also use predefined fill values
fill_values = {"East": 0.5, "West": -1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

Ohio         -0.842303
New York      0.133229
Vermont       0.500000
Florida       1.674211
Oregon        1.102568
Nevada       -1.000000
California   -0.731954
Idaho        -1.000000
dtype: float64

In [63]:
# Example of random sampling and permutation
# Make a deck of cards
suits = ["H", "S", "C", "D"]
card_val = (list(range(1,11)) + [10]*3)*4
base_names = ["A"] + list(range(2,11)) + ["J", "Q", "K"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
QS     10
KS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
QC     10
KC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
QD     10
KD     10
dtype: int64

In [64]:
# Now simulate drawing five cards
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

6C    6
6S    6
3H    3
3D    3
8S    8
dtype: int64

In [65]:
# Draw two cards of the same suit

def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw,n=2)

C  5C     5
   KC    10
D  5D     5
   7D     7
H  4H     4
   9H     9
S  JS    10
   KS    10
dtype: int64

In [66]:
# Compute group weighted averages and correlations
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,1.384545,0.361188
1,a,1.657060,0.683334
2,a,0.108630,0.119894
3,a,0.514719,0.134452
4,b,0.094636,0.552050
5,b,0.352158,0.711195
6,b,1.802832,0.631212
7,b,-1.564878,0.334354


In [67]:
# Compute the weighted average by category
grouped = df.groupby("category")

def get_wavg(group):
    return np.average(group["data"], weights = group["weights"])

grouped.apply(get_wavg)

category
a    1.320100
b    0.411629
dtype: float64